In [ ]:
# ============================================================
# 0. 공통 설정: 드라이브 마운트 + 기본 경로
# ============================================================
!pip install -q opencv-python pandas torch torchvision

from google.colab import drive
drive.mount('/content/drive')

import os, re, zipfile
from pathlib import Path

import cv2
import pandas as pd
import numpy as np
from glob import glob

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T

# ---- 경로 설정 ----
ZIP_PATH = Path("/content/drive/MyDrive/DAiSEE.zip")   # DAiSEE.zip 위치
WORK_ROOT   = Path("/content/workdir")
FRAMES_ROOT = WORK_ROOT / "frames"   # 프레임 저장 루트 (Train/, Validation/ 밑에 clip 폴더)
LISTS_ROOT  = WORK_ROOT / "lists"    # 필요하면 텍스트 리스트 저장
TMP_ROOT    = WORK_ROOT / "tmp"      # 임시 비디오 저장

for d in [FRAMES_ROOT, LISTS_ROOT, TMP_ROOT]:
    d.mkdir(parents=True, exist_ok=True)

print("📁 WORK_ROOT:", WORK_ROOT)
print("📦 ZIP exists:", ZIP_PATH.exists())

# 전체 데이터 다 쓸거니까 1.0
SPLIT_FRACTION = 1.0

Mounted at /content/drive
📁 WORK_ROOT: /content/workdir
📦 ZIP exists: True


In [ ]:
# ============================================================
# 1. DAiSEE 라벨 로드 + 4개 감정(흥미/집중도/이해도/안정감) 생성
# ============================================================
LABEL_CSV_IN_ZIP = "DAiSEE/Labels/AllLabels.csv"

with zipfile.ZipFile(ZIP_PATH) as zf:
    with zf.open(LABEL_CSV_IN_ZIP) as f:
        df_labels = pd.read_csv(f)

print("✅ df_labels 로드 완료")
print(df_labels.columns)
print(df_labels.head())

def make_label_lookup(df_labels):
    # ---- 1) clip id 컬럼 찾기 ----
    cand_clip_cols = [
        "ClipID", "clipID", "Clip Id", "Clip ID",
        "ClipName", "Clip Name", "Clip_Name", "clip_name",
    ]
    clip_col = None
    for c in cand_clip_cols:
        if c in df_labels.columns:
            clip_col = c
            break
    if clip_col is None:
        for c in df_labels.columns:
            low = c.lower()
            if "clip" in low and "id" in low:
                clip_col = c
                break
    if clip_col is None:
        raise ValueError(f"clip id 컬럼을 찾을 수 없음: {df_labels.columns}")

    # ---- 2) 4개 감정 컬럼 찾기 ----
    def find_col(keyword):
        for c in df_labels.columns:
            if keyword in c.lower():
                return c
        return None

    col_eng  = find_col("engagement")
    col_bor  = find_col("boredom")
    col_conf = find_col("confusion")
    col_frus = find_col("frustration")

    if not all([col_eng, col_bor, col_conf, col_frus]):
        raise ValueError(
            f"4개 감정 컬럼 중 일부를 찾을 수 없음: "
            f"eng={col_eng}, bor={col_bor}, conf={col_conf}, frus={col_frus}"
        )

    print(f"🧩 사용 컬럼:")
    print(f"  clip_col    = {clip_col}")
    print(f"  engagement  = {col_eng}")
    print(f"  boredom     = {col_bor}")
    print(f"  confusion   = {col_conf}")
    print(f"  frustration = {col_frus}")

    # DAiSEE 라벨 스케일 (0~3 가정)
    MAX_SCALE = 3

    # ---- 3) lookup[clipid] = {interest, engagement, understanding, stability} ----
    lookup = {}
    for _, row in df_labels.iterrows():
        raw = str(row[clip_col]).strip()
        raw = raw.split(".")[0]  # "1100011002.avi" -> "1100011002"
        m = re.search(r"(\d+)", raw)
        if not m:
            continue
        clipid = m.group(1)

        try:
            eng  = int(row[col_eng])
            bor  = int(row[col_bor])
            conf = int(row[col_conf])
            frus = int(row[col_frus])
        except Exception:
            continue

        # 🔁 여기서 한나가 원하는 4라벨로 "뒤집기" 적용
        interest      = MAX_SCALE - bor        # Boredom 뒤집어서 "흥미"
        engagement    = eng                    # Engagement 그대로 "집중도"
        understanding = MAX_SCALE - conf       # Confusion 뒤집어서 "이해도"
        stability     = MAX_SCALE - frus       # Frustration 뒤집어서 "안정감"

        lookup[clipid] = {
            "interest":      interest,
            "engagement":    engagement,
            "understanding": understanding,
            "stability":     stability,
        }

    print(f"✅ lookup 생성 완료: {len(lookup)}개 (4개 라벨 포함)")
    for i, (k, v) in enumerate(lookup.items()):
        print("  ", k, "->", v)
        if i >= 4:
            break
    return lookup

lookup = make_label_lookup(df_labels)

✅ df_labels 로드 완료
Index(['ClipID', 'Boredom', 'Engagement', 'Confusion', 'Frustration '], dtype='object')
           ClipID  Boredom  Engagement  Confusion  Frustration 
0  1100011002.avi        0           2          0             0
1  1100011003.avi        0           2          0             0
2  1100011004.avi        0           3          0             0
3  1100011005.avi        0           3          0             0
4  1100011006.avi        0           3          0             0
🧩 사용 컬럼:
  clip_col    = ClipID
  engagement  = Engagement
  boredom     = Boredom
  confusion   = Confusion
  frustration = Frustration 
✅ lookup 생성 완료: 8925개 (4개 라벨 포함)
   1100011002 -> {'interest': 3, 'engagement': 2, 'understanding': 3, 'stability': 3}
   1100011003 -> {'interest': 3, 'engagement': 2, 'understanding': 3, 'stability': 3}
   1100011004 -> {'interest': 3, 'engagement': 3, 'understanding': 3, 'stability': 3}
   1100011005 -> {'interest': 3, 'engagement': 3, 'understanding': 3, 'stability'

In [ ]:
# ============================================================
# 2. ZIP 안 영상 → 프레임 추출 (Train / Validation)
#    BadZipFile 무시 + 이미 추출된 프레임 있으면 스킵
# ============================================================
def video_from_zip_to_frames(zf: zipfile.ZipFile, member_name: str,
                             out_dir: Path, resize=(224, 224)) -> int:
    """
    ZIP 안의 단일 .avi 멤버를 프레임으로 추출.
    out_dir: /content/workdir/frames/Train/1100011002 같은 폴더
    이미 프레임이 있다면 건너뜀.
    """
    # 이미 추출된 경우 스킵
    if out_dir.exists() and any(out_dir.glob("img_*.jpg")):
        # 이미 만들어둔 프레임 개수 리턴
        return len(list(out_dir.glob("img_*.jpg")))

    out_dir.mkdir(parents=True, exist_ok=True)
    TMP_ROOT.mkdir(parents=True, exist_ok=True)
    tmp_path = TMP_ROOT / "tmp_video.avi"

    # ZIP → 로컬 임시 .avi
    try:
        with zf.open(member_name) as src, open(tmp_path, "wb") as dst:
            # 한 번에 다 read() 말고, chunk 단위로 읽으면 안정적
            for chunk in iter(lambda: src.read(1024 * 1024), b""):
                if not chunk:
                    break
                dst.write(chunk)
    except zipfile.BadZipFile:
        print(f"⚠️ BadZipFile, 이 영상은 건너뜀: {member_name}")
        tmp_path.unlink(missing_ok=True)
        return 0
    except Exception as e:
        print(f"⚠️ ZIP 읽기 중 에러, 건너뜀: {member_name} ({e})")
        tmp_path.unlink(missing_ok=True)
        return 0

    cap = cv2.VideoCapture(str(tmp_path))
    if not cap.isOpened():
        print(f"❌ Cannot open video: {member_name}")
        tmp_path.unlink(missing_ok=True)
        return 0

    frame_idx = 0
    saved = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame_idx += 1

        frame = cv2.resize(frame, resize)
        out_file = out_dir / f"img_{saved+1:05d}.jpg"
        cv2.imwrite(str(out_file), frame)
        saved += 1

    cap.release()
    tmp_path.unlink(missing_ok=True)
    return saved


with zipfile.ZipFile(ZIP_PATH) as zf:
    all_members = zf.namelist()
    for SPLIT in ["Train", "Validation"]:
        print(f"\n==================== {SPLIT} 프레임 추출 시작 ====================")
        split_tag = f"DAiSEE/DataSet/{SPLIT}/"
        split_members = [m for m in all_members if m.endswith(".avi") and split_tag in m]
        split_members = sorted(split_members)
        total_videos = len(split_members)
        print(f"📦 {SPLIT} .avi 개수: {total_videos}")

        if total_videos == 0:
            continue

        use_count = max(1, int(total_videos * SPLIT_FRACTION))
        split_members = split_members[:use_count]
        print(f"⏱ {SPLIT}: {use_count}/{total_videos}개 사용 (fraction={SPLIT_FRACTION})")

        processed = 0
        total_frames = 0

        for member_name in split_members:
            clipid = Path(member_name).stem  # "1100011002"
            out_dir = FRAMES_ROOT / SPLIT / clipid

            # 혹시 함수 밖에서 또 한 번 방어하고 싶으면 try/except 한 번 더
            try:
                n_frames = video_from_zip_to_frames(zf, member_name, out_dir)
            except Exception as e:
                print(f"⚠️ 알 수 없는 에러로 {member_name} 스킵: {e}")
                continue

            if n_frames > 0:
                processed += 1
                total_frames += n_frames

            if processed % 20 == 0:
                print(f"  - 진행: {processed}/{use_count}개, 누적 프레임 {total_frames}")

        print(f"✅ {SPLIT} 프레임 추출 완료: {processed}개 영상, {total_frames}장 프레임")


==================== Train 프레임 추출 시작 ====================
📦 Train .avi 개수: 4976
⏱ Train: 4976/4976개 사용 (fraction=1.0)
  - 진행: 20/4976개, 누적 프레임 6000
  - 진행: 40/4976개, 누적 프레임 12000
  - 진행: 60/4976개, 누적 프레임 18000
  - 진행: 80/4976개, 누적 프레임 24000
  - 진행: 100/4976개, 누적 프레임 30000
  - 진행: 120/4976개, 누적 프레임 36000
  - 진행: 140/4976개, 누적 프레임 42000
  - 진행: 160/4976개, 누적 프레임 48000
  - 진행: 180/4976개, 누적 프레임 54000
  - 진행: 200/4976개, 누적 프레임 60000
  - 진행: 220/4976개, 누적 프레임 66000
  - 진행: 240/4976개, 누적 프레임 72000
  - 진행: 260/4976개, 누적 프레임 78000
  - 진행: 280/4976개, 누적 프레임 84000
  - 진행: 300/4976개, 누적 프레임 90000
  - 진행: 320/4976개, 누적 프레임 96000
  - 진행: 340/4976개, 누적 프레임 102000
  - 진행: 360/4976개, 누적 프레임 108000
  - 진행: 380/4976개, 누적 프레임 114000
  - 진행: 400/4976개, 누적 프레임 120000
  - 진행: 420/4976개, 누적 프레임 126000
  - 진행: 440/4976개, 누적 프레임 132000
  - 진행: 460/4976개, 누적 프레임 138000
  - 진행: 480/4976개, 누적 프레임 144000
  - 진행: 500/4976개, 누적 프레임 150000
  - 진행: 520/4976개, 누적 프레임 156000
  - 진행: 540/4976개, 누적 프레임 162000
  - 진행: 560

In [ ]:
# ============================================================
# 3. Train/Val 샘플 리스트 만들기 (한 모델이 4라벨 동시에 예측)
# ============================================================
TARGET_LABELS = ["interest", "engagement", "understanding", "stability"]

def build_samples(split_name: str, frames_root: Path, lookup: dict):
    """
    반환: samples = [
      {
        "dir": clip_dir(Path),
        "n_frames": int,
        "labels": [interest, engagement, understanding, stability]
      }, ...
    ]
    """
    split_root = frames_root / split_name
    if not split_root.exists():
        print(f"⚠️ {split_name} 프레임 폴더 없음: {split_root}")
        return []

    samples = []
    for d in sorted(split_root.iterdir()):
        if not d.is_dir():
            continue
        clipid = d.name
        label_dict = lookup.get(clipid)
        if label_dict is None:
            continue

        frame_files = sorted(d.glob("img_*.jpg"))
        n_frames = len(frame_files)
        if n_frames == 0:
            continue

        labels = [int(label_dict[k]) for k in TARGET_LABELS]
        samples.append({
            "dir": d,
            "n_frames": n_frames,
            "labels": labels
        })

    print(f"✅ {split_name} 샘플 개수: {len(samples)}")
    return samples

train_samples = build_samples("Train", FRAMES_ROOT, lookup)
val_samples   = build_samples("Validation", FRAMES_ROOT, lookup)

print("예시 train sample:", train_samples[0])

✅ Train 샘플 개수: 4975
✅ Validation 샘플 개수: 1536
예시 train sample: {'dir': PosixPath('/content/workdir/frames/Train/1100011002'), 'n_frames': 300, 'labels': [3, 2, 3, 3]}


In [ ]:
# ============================================================
# 4. PyTorch Dataset 정의 (16프레임 샘플링 + 4라벨 반환)
# ============================================================
class MultiLabelVideoDataset(Dataset):
    def __init__(self, samples, num_segments=16, transform=None):
        self.samples = samples
        self.num_segments = num_segments
        self.transform = transform

    def __len__(self):
        return len(self.samples)

    def _sample_indices(self, n_frames):
        """프레임 수 n_frames에서 num_segments개를 균등 샘플링"""
        if n_frames < self.num_segments:
            # 부족하면 반복해서 채우기
            idx = np.linspace(0, n_frames - 1, n_frames, dtype=int)
            idx = np.concatenate([idx, np.random.choice(idx, self.num_segments - n_frames)])
        else:
            idx = np.linspace(0, n_frames - 1, self.num_segments, dtype=int)
        return idx

    def __getitem__(self, idx):
        item = self.samples[idx]
        clip_dir = item["dir"]
        n_frames = item["n_frames"]
        labels   = torch.tensor(item["labels"], dtype=torch.long)

        indices = self._sample_indices(n_frames)
        frames = []
        for i in indices:
            frame_path = clip_dir / f"img_{i+1:05d}.jpg"
            img = cv2.imread(str(frame_path))
            if img is None:
                # 혹시 빠진 프레임 있으면 첫 프레임으로 대체
                img = cv2.imread(str(clip_dir / "img_00001.jpg"))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            if self.transform is not None:
                img = self.transform(img)
            else:
                img = torch.from_numpy(img).permute(2,0,1).float() / 255.0
            frames.append(img)

        # [T, C, H, W] → [C, T, H, W] or [N, T, C, H, W] 형태로 쓸 수 있음
        frames = torch.stack(frames, dim=0)  # [T, C, H, W]
        return frames, labels


# 이미지 변환 (기본적인 normalize)
transform = T.Compose([
    T.ToPILImage(),
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225]),
])

NUM_SEGMENTS = 16
BATCH_SIZE   = 4

train_dataset = MultiLabelVideoDataset(train_samples, num_segments=NUM_SEGMENTS, transform=transform)
val_dataset   = MultiLabelVideoDataset(val_samples,   num_segments=NUM_SEGMENTS, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_dataset,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print("train dataset len:", len(train_dataset))
print("val dataset len:", len(val_dataset))

train dataset len: 4975
val dataset len: 1536


In [ ]:
# ============================================================
# 5. 멀티헤드 TSN 스타일 모델 정의
#    - backbone: ResNet50 (ImageNet 사전학습)
#    - 시간 방향: frame별 feature 뽑아서 평균(pooling)
#    - head 4개: 흥미 / 집중도 / 이해도 / 안정감
# ============================================================
import torchvision.models as models

class MultiHeadTSN(nn.Module):
    def __init__(self, num_segments=16, num_classes_per_head=4):
        super().__init__()
        base = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
        feat_dim = base.fc.in_features
        base.fc = nn.Identity()   # 마지막 fc 제거해서 feature만 뽑도록
        self.base = base
        self.feat_dim = feat_dim
        self.num_segments = num_segments

        # 4개의 head (각 4-class 분류)
        self.head_interest      = nn.Linear(feat_dim, num_classes_per_head)
        self.head_engagement    = nn.Linear(feat_dim, num_classes_per_head)
        self.head_understanding = nn.Linear(feat_dim, num_classes_per_head)
        self.head_stability     = nn.Linear(feat_dim, num_classes_per_head)

    def forward(self, x):
        """
        x: [N, T, C, H, W]
        """
        N, T, C, H, W = x.shape
        assert T == self.num_segments, f"num_segments={self.num_segments}, but got T={T}"

        x = x.view(N * T, C, H, W)
        feat = self.base(x)              # [N*T, feat_dim]
        feat = feat.view(N, T, self.feat_dim).mean(dim=1)  # 시간 평균 → [N, feat_dim]

        out_interest      = self.head_interest(feat)
        out_engagement    = self.head_engagement(feat)
        out_understanding = self.head_understanding(feat)
        out_stability     = self.head_stability(feat)

        return {
            "interest":      out_interest,
            "engagement":    out_engagement,
            "understanding": out_understanding,
            "stability":     out_stability,
        }

In [ ]:
# ============================================================
# 6. 학습 루프 + 체크포인트 저장
# ============================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("사용 디바이스:", device)

model = MultiHeadTSN(num_segments=NUM_SEGMENTS, num_classes_per_head=4).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
num_epochs = 20   # 필요하면 늘리기

CKPT_ROOT = WORK_ROOT / "checkpoints"
CKPT_ROOT.mkdir(parents=True, exist_ok=True)

def train_one_epoch(epoch, model, loader, optimizer, device):
    model.train()
    running_loss = 0.0
    total = 0

    for i, (frames, labels) in enumerate(loader):
        # frames: [B, T, C, H, W]
        frames = frames.to(device)
        labels = labels.to(device)   # [B, 4]

        optimizer.zero_grad()
        outputs = model(frames)

        # 4라벨 각각 CE loss 계산
        loss_interest      = criterion(outputs["interest"],      labels[:, 0])
        loss_engagement    = criterion(outputs["engagement"],    labels[:, 1])
        loss_understanding = criterion(outputs["understanding"], labels[:, 2])
        loss_stability     = criterion(outputs["stability"],     labels[:, 3])

        loss = (loss_interest + loss_engagement + loss_understanding + loss_stability) / 4.0
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * frames.size(0)
        total += frames.size(0)

        if (i + 1) % 10 == 0:
            print(f"  [Epoch {epoch+1} Iter {i+1}/{len(loader)}] loss={loss.item():.4f}")

    epoch_loss = running_loss / total
    return epoch_loss

@torch.no_grad()
def evaluate(model, loader, device):
    model.eval()
    total = 0
    correct_interest = 0
    correct_engagement = 0
    correct_understanding = 0
    correct_stability = 0

    for frames, labels in loader:
        frames = frames.to(device)
        labels = labels.to(device)

        outputs = model(frames)
        # 각 head별 argmax
        pred_interest      = outputs["interest"].argmax(dim=1)
        pred_engagement    = outputs["engagement"].argmax(dim=1)
        pred_understanding = outputs["understanding"].argmax(dim=1)
        pred_stability     = outputs["stability"].argmax(dim=1)

        total += frames.size(0)
        correct_interest      += (pred_interest      == labels[:, 0]).sum().item()
        correct_engagement    += (pred_engagement    == labels[:, 1]).sum().item()
        correct_understanding += (pred_understanding == labels[:, 2]).sum().item()
        correct_stability     += (pred_stability     == labels[:, 3]).sum().item()

    acc_interest      = correct_interest      / total
    acc_engagement    = correct_engagement    / total
    acc_understanding = correct_understanding / total
    acc_stability     = correct_stability     / total

    return {
        "interest":      acc_interest,
        "engagement":    acc_engagement,
        "understanding": acc_understanding,
        "stability":     acc_stability,
    }

best_val_avg = 0.0

for epoch in range(num_epochs):
    print(f"\n===== Epoch {epoch+1}/{num_epochs} =====")
    train_loss = train_one_epoch(epoch, model, train_loader, optimizer, device)
    print(f"📉 Train Loss: {train_loss:.4f}")

    val_accs = evaluate(model, val_loader, device)
    avg_acc = np.mean(list(val_accs.values()))
    print(f"📊 Val Acc - 흥미:{val_accs['interest']:.3f}, "
          f"집중도:{val_accs['engagement']:.3f}, "
          f"이해도:{val_accs['understanding']:.3f}, "
          f"안정감:{val_accs['stability']:.3f}, "
          f"평균:{avg_acc:.3f}")

    # ✅ 매 epoch마다 체크포인트 저장
    ckpt_path = CKPT_ROOT / f"tsn_multilabel_epoch{epoch+1}.pth"
    torch.save({
        "epoch": epoch + 1,
        "model_state": model.state_dict(),
        "optimizer_state": optimizer.state_dict(),
        "val_accs": val_accs,
    }, ckpt_path)
    print("💾 체크포인트 저장:", ckpt_path)

    # 베스트 모델 따로 저장
    if avg_acc > best_val_avg:
        best_val_avg = avg_acc
        best_path = CKPT_ROOT / "tsn_multilabel_best.pth"
        torch.save({
            "epoch": epoch + 1,
            "model_state": model.state_dict(),
            "optimizer_state": optimizer.state_dict(),
            "val_accs": val_accs,
        }, best_path)
        print("🌟 베스트 모델 갱신:", best_path, " (avg acc =", best_val_avg, ")")

사용 디바이스: cuda
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 212MB/s]



===== Epoch 1/20 =====
  [Epoch 1 Iter 10/1244] loss=1.1774
  [Epoch 1 Iter 20/1244] loss=0.9717
  [Epoch 1 Iter 30/1244] loss=0.5279
  [Epoch 1 Iter 40/1244] loss=1.0142
  [Epoch 1 Iter 50/1244] loss=1.2740
  [Epoch 1 Iter 60/1244] loss=1.0226
  [Epoch 1 Iter 70/1244] loss=0.9272
  [Epoch 1 Iter 80/1244] loss=0.6982
  [Epoch 1 Iter 90/1244] loss=0.6045
  [Epoch 1 Iter 100/1244] loss=0.8548
  [Epoch 1 Iter 110/1244] loss=0.8854
  [Epoch 1 Iter 120/1244] loss=1.0815
  [Epoch 1 Iter 130/1244] loss=1.0220
  [Epoch 1 Iter 140/1244] loss=0.6014
  [Epoch 1 Iter 150/1244] loss=0.8643
  [Epoch 1 Iter 160/1244] loss=1.1135
  [Epoch 1 Iter 170/1244] loss=0.7896
  [Epoch 1 Iter 180/1244] loss=1.0826
  [Epoch 1 Iter 190/1244] loss=1.1752
  [Epoch 1 Iter 200/1244] loss=0.6972
  [Epoch 1 Iter 210/1244] loss=0.9645
  [Epoch 1 Iter 220/1244] loss=0.6184
  [Epoch 1 Iter 230/1244] loss=0.6927
  [Epoch 1 Iter 240/1244] loss=1.0397
  [Epoch 1 Iter 250/1244] loss=1.0976
  [Epoch 1 Iter 260/1244] loss=0.75

In [ ]:
import collections

interest_counts      = collections.Counter([s["labels"][0] for s in train_samples])
engagement_counts    = collections.Counter([s["labels"][1] for s in train_samples])
understanding_counts = collections.Counter([s["labels"][2] for s in train_samples])
stability_counts     = collections.Counter([s["labels"][3] for s in train_samples])

print("흥미(interest) 분포:", interest_counts)
print("집중도(engagement) 분포:", engagement_counts)
print("이해도(understanding) 분포:", understanding_counts)
print("안정감(stability) 분포:", stability_counts)

흥미(interest) 분포: Counter({3: 2236, 2: 1547, 1: 1044, 0: 148})
집중도(engagement) 분포: Counter({2: 2506, 3: 2234, 1: 202, 0: 33})
이해도(understanding) 분포: Counter({3: 3406, 2: 1103, 1: 405, 0: 61})
안정감(stability) 분포: Counter({3: 3893, 2: 854, 1: 188, 0: 40})
